In [93]:
!poetry add pyarrow --dev

Using version ^2.0.0 for pyarrow

Updating dependencies
Resolving dependencies... (1.9s)

Writing lock file

Package operations: 1 install, 0 updates, 0 removals

  • Installing pyarrow (2.0.0): Pending...
  • Installing pyarrow (2.0.0): Downloading... 0%
  • Installing pyarrow (2.0.0): Downloading... 10%
  • Installing pyarrow (2.0.0): Downloading... 20%
  • Installing pyarrow (2.0.0): Downloading... 30%
  • Installing pyarrow (2.0.0): Downloading... 40%
  • Installing pyarrow (2.0.0): Downloading... 50%
  • Installing pyarrow (2.0.0): Downloading... 60%
  • Installing pyarrow (2.0.0): Downloading... 70%
  • Installing pyarrow (2.0.0): Downloading... 80%
  • Installing pyarrow (2.0.0): Downloading... 90%
  • Installing pyarrow (2.0.0): Downloading... 100%
  • Installing pyarrow (2.0.0): Downloading... 100%
  • Installing pyarrow (2.0.0): Installing...
  • Installing pyarrow (2.0.0)


In [1]:
import requests

from pathlib import Path
import json
import time

import pyarrow.json
import pyarrow.parquet
import pandas as pd

In [73]:
def get_table_properties(table_url):
    r = requests.get(table_url +"/Properties").json()
    return r

In [2]:
def get_odata_pd(target_url):
    data = pd.DataFrame()
    while target_url:
        r = requests.get(target_url).json()
        data = data.append(pd.DataFrame(r['value']))
        
        if '@odata.nextLink' in r:
            target_url = r['@odata.nextLink']
        else:
            target_url = None
            
    return data

In [3]:
def get_odata_list(target_url):
    data = []
    while target_url:
        r = requests.get(target_url).json()
        data.extend(r['value'])
        
        if '@odata.nextLink' in r:
            target_url = r['@odata.nextLink']
        else:
            target_url = None
            
    return data

In [21]:
TEMP = "test-cbs-v4-to-parquet"
TEMP_JSON = Path(TEMP + ".json")
TEMP_NDJSON = Path(TEMP + ".ndjson")
TEMP_PARQUET = Path(TEMP + ".parquet")

TESTDATA = "82807NED"

In [22]:
table_url = f"https://odata4.cbs.nl/CBS/{TESTDATA}"
target_url = table_url + "/Observations"
# data_82931 = get_odata(target_url)

In [62]:
# List all tables under a dataset ID
meta = get_odata_pd(table_url)
meta

,name,kind,url
0,MeasureCodes,EntitySet,MeasureCodes
1,Dimensions,EntitySet,Dimensions
2,PeriodenGroups,EntitySet,PeriodenGroups
3,PeriodenCodes,EntitySet,PeriodenCodes
4,BedrijfstakkenBranchesSBI2008Groups,EntitySet,BedrijfstakkenBranchesSBI2008Groups
5,BedrijfstakkenBranchesSBI2008Codes,EntitySet,BedrijfstakkenBranchesSBI2008Codes
6,PersoonskenmerkenGroups,EntitySet,PersoonskenmerkenGroups
7,PersoonskenmerkenCodes,EntitySet,PersoonskenmerkenCodes
8,GeslachtCodes,EntitySet,GeslachtCodes
9,Observations,EntitySet,Observations


In [47]:
# Get all metadata table names (=url extensions).
# Exclude Observations as it is the actual data and retrieved separately
# Also include Properties as it is not relevant
meta_names = [url for url in meta.url.tolist() if url not in ['Observations', 'Properties']]
meta_names

['MeasureCodes',
 'Dimensions',
 'PeriodenGroups',
 'PeriodenCodes',
 'BedrijfstakkenBranchesSBI2008Groups',
 'BedrijfstakkenBranchesSBI2008Codes',
 'PersoonskenmerkenGroups',
 'PersoonskenmerkenCodes',
 'GeslachtCodes']

In [50]:
# Convert all metadata names to actual urls for table retrieval
meta_urls = [table_url + "/" + name for name in meta_names]
meta_urls

['https://odata4.cbs.nl/CBS/82807NED/MeasureCodes',
 'https://odata4.cbs.nl/CBS/82807NED/Dimensions',
 'https://odata4.cbs.nl/CBS/82807NED/PeriodenGroups',
 'https://odata4.cbs.nl/CBS/82807NED/PeriodenCodes',
 'https://odata4.cbs.nl/CBS/82807NED/BedrijfstakkenBranchesSBI2008Groups',
 'https://odata4.cbs.nl/CBS/82807NED/BedrijfstakkenBranchesSBI2008Codes',
 'https://odata4.cbs.nl/CBS/82807NED/PersoonskenmerkenGroups',
 'https://odata4.cbs.nl/CBS/82807NED/PersoonskenmerkenCodes',
 'https://odata4.cbs.nl/CBS/82807NED/GeslachtCodes']

In [51]:
# Get observations data, meaning the actual values of the dataset
observations = get_odata_pd(target_url)
observations

,Id,Measure,ValueAttribute,Value,Perioden,BedrijfstakkenBranchesSBI2008,Persoonskenmerken,Geslacht
0,0,3000795_1,None,7771.0,2003KW01,T001081,T009002,T001038
1,1,3000795_1,None,7782.0,2003KW02,T001081,T009002,T001038
2,2,3000795_1,None,7809.0,2003KW03,T001081,T009002,T001038
3,3,3000795_1,None,7770.0,2003KW04,T001081,T009002,T001038
4,4,3000795_1,None,7783.0,2003JJ00,T001081,T009002,T001038
...,...,...,...,...,...,...,...,...
81765,496075,3000795_1,None,0.0,2018KW01,999999,2021440,4000
81766,496076,3000795_1,None,0.0,2018KW02,999999,2021440,4000
81767,496077,3000795_1,None,0.0,2018KW03,999999,2021440,4000
81768,496078,3000795_1,None,0.0,2018KW04,999999,2021440,4000


In [53]:
meta_tables = [get_odata_pd(url) for url in meta_urls]

In [59]:
meta_tables[3]

,Identifier,Index,Title,Description,DimensionGroupId,Status
0,2003KW01,1,2003 1e kwartaal,,0,Definitief
1,2003KW02,2,2003 2e kwartaal,,0,Definitief
2,2003KW03,3,2003 3e kwartaal,,0,Definitief
3,2003KW04,4,2003 4e kwartaal,,0,Definitief
4,2003JJ00,5,2003,,1,Definitief
...,...,...,...,...,...,...
75,2018KW01,76,2018 1e kwartaal,,0,Definitief
76,2018KW02,77,2018 2e kwartaal,,0,Definitief
77,2018KW03,78,2018 3e kwartaal,,0,Definitief
78,2018KW04,79,2018 4e kwartaal,,0,Definitief


In [58]:
meta_tables[2]

,Id,Index,Title,Description,ParentId
0,1,0,Jaren,None,None
1,0,2,Kwartalen,None,None


In [26]:
# list all dataset dimensions
dimensions = get_odata_pd(table_url + "/Dimensions")
dimensions

,Identifier,Title,Description,Kind,MapYear,ReleasePolicy
0,Perioden,Perioden,,TimeDimension,None,True
1,BedrijfstakkenBranchesSBI2008,Bedrijfstakken/branches SBI 2008,,Dimension,None,None
2,Persoonskenmerken,Persoonskenmerken,,Dimension,None,None
3,Geslacht,Geslacht,,Dimension,None,None


In [ ]:
[for dim in dimensions]

In [30]:
# get Measure Codes
codes = get_odata_pd(table_url + "/MeasureCodes")
codes

,Identifier,Index,Title,Description,MeasureGroupId,DataType,Unit,Format,Decimals,PresentationType
0,3000795_1,4,Werkzame beroepsbevolking,Personen die betaald werk hebben.\r\n\r\nDeze ...,None,Integer,x 1 000,None,0,None


In [35]:
codes.Description

0    Personen die betaald werk hebben.\r\n\r\nDeze ...
Name: Description, dtype: object

In [17]:
cbs_catalogs = get_odata_pd("https://odata4.cbs.nl/CBS/Catalogs")
cbs_datasets = get_odata_pd("https://odata4.cbs.nl/CBS/Datasets")

In [20]:
cbs_datasets

,Identifier,Description,Language,Title,Modified,Catalog,Version,VersionNotes,VersionReason,Status,ObservationsModified,ObservationCount,DatasetType
0,80784ned,\nDeze tabel bevat gegevens op regioniveau ove...,nl,Landbouw; arbeidskrachten naar regio,2020-03-03T02:00:00+01:00,CBS,202003030200,,Actualisering,Regulier,2020-03-03T02:00:00+01:00,173148,Numeric
1,81075ned,\nDeze tabel gaat over de werkloosheidsduur va...,nl,Werkloze beroepsbevolking; werkloosheidsduur e...,2020-01-23T02:00:00+01:00,CBS,202001230200,,Stopgezet,Gediscontinueerd,2019-11-14T02:00:00+01:00,2432,Numeric
2,81575NED,\nDeze tabel bevat gegevens over het aantal ve...,nl,"Vestigingen van bedrijven; bedrijfstak, gemeente",2020-04-08T02:00:00+02:00,CBS,202004080200,,Actualisering,Regulier,2020-04-08T02:00:00+02:00,142800,Numeric
3,82245NED,\nDeze tabel bevat cijfers over de Nederlandse...,nl,Bevolking en huishoudens; viercijferige postco...,2019-08-23T02:00:00+02:00,CBS,201908230200,,Stopgezet,Gediscontinueerd,2013-11-15T02:00:00+01:00,326174,Numeric
4,82807NED,\nDeze tabel bevat kwartaal- en jaarcijfers ov...,nl,Werkzame beroepsbevolking; bedrijf,2019-11-29T02:00:00+01:00,CBS,201911290200,De cijfers over het onderwijsniveau in deze ta...,Correctie,Regulier,2019-11-29T02:00:00+01:00,481770,Numeric
5,82931NED,\nOverzicht van statistische gegevens van geme...,nl,Kerncijfers wijken en buurten 2014,2018-07-27T02:00:00+02:00,CBS,201807270200,,Verbetering,Regulier,2018-07-27T02:00:00+02:00,1348572,Numeric
6,83220NED,\nOverzicht van statistische gegevens van geme...,nl,Kerncijfers wijken en buurten 2015,2018-07-27T02:00:00+02:00,CBS,201807270200,,Verbetering,Regulier,2018-07-27T02:00:00+02:00,1401105,Numeric
7,83433NED,\nDeze tabel bevat cijfers over het prijsverlo...,nl,"Consumentenprijzen; werknemers laag, alle basi...",2016-04-12T02:00:00+02:00,CBS,201604120200,,Nieuw,Gediscontinueerd,2016-04-12T02:00:00+02:00,2580,Numeric
8,83435NED,\nDeze tabel bevat cijfers over het aantal pas...,nl,"Amsterdam Airport Schiphol; passagiersvervoer,...",2020-02-14T02:00:00+01:00,CBS,202002140200,,Actualisering,Regulier,2020-02-14T02:00:00+01:00,28742,Numeric
9,83487NED,\nOverzicht van statistische gegevens van geme...,nl,Kerncijfers wijken en buurten 2016,2019-02-01T02:00:00+01:00,CBS,201902010200,,Actualisering bijzonder,Regulier,2019-02-01T02:00:00+01:00,1511607,Numeric


In [18]:
ids = cbs_datasets.Identifier.tolist()

In [19]:
all_table_urls = ["https://odata4.cbs.nl/CBS/"+id for id in ids]

In [134]:
df = pd.DataFrame()
for url in all_table_urls:
    table = get_odata_pd(url)
    df = df.append(table)

In [68]:
table_names = [table['url'] for table in get_odata(table_url) if table['url']!='Properties']
table_names

['MeasureCodes',
 'Dimensions',
 'PeriodenGroups',
 'PeriodenCodes',
 'BedrijfstakkenBranchesSBI2008Codes',
 'Observations']

In [69]:
table_urls = [table_url + "/" + name for name in table_names]
table_urls

['https://odata4.cbs.nl/CBS/84287NED/MeasureCodes',
 'https://odata4.cbs.nl/CBS/84287NED/Dimensions',
 'https://odata4.cbs.nl/CBS/84287NED/PeriodenGroups',
 'https://odata4.cbs.nl/CBS/84287NED/PeriodenCodes',
 'https://odata4.cbs.nl/CBS/84287NED/BedrijfstakkenBranchesSBI2008Codes',
 'https://odata4.cbs.nl/CBS/84287NED/Observations']

In [70]:
tables = [get_odata(url) for url in table_urls]

In [72]:
tables_dict.keys()

dict_keys(['MeasureCodes', 'Dimensions', 'PeriodenGroups', 'PeriodenCodes', 'BedrijfstakkenBranchesSBI2008Codes', 'Observations'])

In [74]:
tables_dict['Dimensions']

[{'Identifier': 'Perioden',
  'Title': 'Perioden',
  'Description': '',
  'Kind': 'TimeDimension',
  'MapYear': None,
  'ReleasePolicy': 'True'},
 {'Identifier': 'BedrijfstakkenBranchesSBI2008',
  'Title': 'Bedrijfstakken/branches (SBI 2008)',
  'Description': '',
  'Kind': 'Dimension',
  'MapYear': None,
  'ReleasePolicy': None}]

In [71]:
tables_dict = dict(zip(table_names, tables))

In [14]:
with open(TEMP_NDJSON, "w+") as ndjson:
    for record in table:
        ndjson.write(json.dumps(record) + "\n")